In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import os
from tqdm import tqdm, trange
from polyleven import levenshtein
import RNA

In [2]:
def read_fasta_sequences(fasta_file):
    """
    读取FASTA文件并返回序列列表。
    
    :param fasta_file: FASTA文件的路径
    :return: 序列列表
    """
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
    return sequences

def analyze_mfe_data(gens):
    # 计算生成数据的MFE
    mfe = []
    ss = []
    for seq in tqdm(gens):
        ss_, mfe_ = RNA.fold(seq)
        mfe.append(mfe_)
        ss.append(ss_)
    return ss, mfe


def hamming_dist(src, target):
    
    dists = []
    for i in trange(len(src)):
        smallest = np.inf
        for j in range(len(target)):
            dist = levenshtein(src[i],target[j])
            if dist > 0 and dist < smallest:
                smallest = dist

        dists.append(smallest)

    return np.array(dists)

def hamming_dist_idx(src, target):
    dists = []
    indices = []
    
    for i in trange(len(src)):
        smallest = np.inf
        src_idx = i
        target_idx = None
        
        for j in range(len(target)):
            dist = levenshtein(src[i], target[j])
            if dist > 0 and dist < smallest:
                smallest = dist
                target_idx = j

        dists.append(smallest)
        indices.append((src_idx, target_idx))

    return np.array(dists), indices

def get_sorted_extremes(data, n = 5):
    sorted_indices = np.argsort(data)
    first_n_indices = sorted_indices[:n]
    return first_n_indices

def write_to_fasta(data, dist_col = 'SSDist_RF02531_Reals45'):
    seqs = list(data.sequence)
    dist = list(data[dist_col])
    
    fasta = ''
    for s, d in zip(seqs, dist):
        fasta += f'>Dist{d}\n{s}\n'
    fasta = fasta[:-1]
    
    fn = f'vAug23_{dist_col}_Top5_FilterVJun8_vMay14.12_vMay14.10_Gene_Class1_GeneNum2000.fasta'
    with open(f'./Top5_IRES_family_Gens/{fn}', 'w') as f:
        f.write(fasta)
        
    family = dist_col.split('_')[1]
    print(f"cmalign --outformat Stockholm /Users/yanyichu/Downloads/IRES_family_fasta/{family}.cm /Users/yanyichu/Downloads/Top5_IRES_family_Gens/{fn} > /Users/yanyichu/Downloads/Top5_IRES_family_Gens/{fn[:-6]}.sto")

# IRESsite

In [16]:
iressite_data = pd.read_csv('IRESsite_shorterthan300bp.csv')
iressite_seq = list(iressite_data['Sequence'])
iressite_id = list(iressite_data['ID'])
iressite_ss, iressite_mfe = analyze_mfe_data(iressite_seq)
iressite_data['SS'] = iressite_ss
iressite_data['MFE'] = iressite_mfe
iressite_data.to_csv('IRESsite_shorterthan300bp_SS_MFE.csv')

100%|███████████████████████████████████████████| 81/81 [00:04<00:00, 16.73it/s]


# Reward Guided

In [11]:
gen_data_1412 = pd.read_csv('../results/IRESDM_RewardGuided_CLSutrlm_Gene_Class1_correctNum6178_Length25+_ep149_SSMFE.csv', index_col = 0)
gen_ss_1412 = gen_data_1412['SS'] 
gen_mfe_1412 = gen_data_1412['MFE']
gen_seq_1412 = list(gen_data_1412.sequence)
gen_data_1412.head()

,sequence,length,Prob_Mean,Pred_Mean,Pred_U_Mean,Pred_R_Mean,Prob_U_Mean,Prob_R_Mean,Prob_U0,Prob_U1,...,Pred_R2,Pred_R3,Pred_R4,Pred_R5,Pred_R6,Pred_R7,Pred_R8,Pred_R9,SS,MFE
0,CACTGTCCAATTCACTTACTCAAGT,25,0.835365,1.0000,1.0,1.0,0.930464,0.778306,0.930259,0.974096,...,1,1,1,1,1,1,1,1,.........................,0.0
1,AGAGGTGATACTCCAATTCTCTTAC,25,0.905741,1.0000,1.0,1.0,0.975414,0.863938,0.988113,0.978957,...,1,1,1,1,1,1,1,1,.(((......)))............,-2.6
2,GCCGACCGGTTTCAACTCCCCGTTT,25,0.883361,1.0000,1.0,1.0,0.950916,0.842828,0.927866,0.973609,...,1,1,1,1,1,1,1,1,(((....)))...............,-1.3
3,GGCCACCATTTCTCTCACAGTTGAT,25,0.869169,1.0000,1.0,1.0,0.951694,0.819654,0.884784,0.973977,...,1,1,1,1,1,1,1,1,.........................,0.0
4,TGCATTCCGAAATTATTTGACAAGT,25,0.835800,0.9375,1.0,0.9,0.927583,0.780731,0.988046,0.960093,...,0,1,1,1,1,1,1,1,.........................,0.0


# Directed Training

In [12]:
gen_data_1410 = pd.read_csv('../results/IRESDM_DirectedTraining_Gene_Class1_correctNum6076_Length25+_ep149_SSMFE.csv', index_col = 0)
gen_ss_1410 = gen_data_1410['SS'] 
gen_mfe_1410 = gen_data_1410['MFE']
gen_seq_1410 = list(gen_data_1410.sequence)
gen_data_1410.head()

,sequence,length,Prob_Mean,Pred_Mean,Pred_U_Mean,Pred_R_Mean,Prob_U_Mean,Prob_R_Mean,Prob_U0,Prob_U1,...,Pred_R2,Pred_R3,Pred_R4,Pred_R5,Pred_R6,Pred_R7,Pred_R8,Pred_R9,SS,MFE
0,AACCGGCGGTCATTTTGTGTCTTAT,25,0.899836,1.000,1.000000,1.0,0.975917,0.854187,0.979265,0.991897,...,1,1,1,1,1,1,1,1,....((((.........))))....,-1.5
1,AGAGGTGATACTCCAATTCTTTTAA,25,0.913532,1.000,1.000000,1.0,0.962951,0.883881,0.985999,0.975400,...,1,1,1,1,1,1,1,1,.(((......)))............,-2.6
2,CATCCCAGCTCCTTCTCGTTATCTA,25,0.898681,1.000,1.000000,1.0,0.970309,0.855705,0.960865,0.988776,...,1,1,1,1,1,1,1,1,.........................,0.0
3,TGGGGTACCACAAGGCCCATACTCC,25,0.786517,1.000,1.000000,1.0,0.893442,0.722362,0.879990,0.783565,...,1,1,1,1,1,1,1,1,..((((........)))).......,-5.6
4,TTAACGTATGAAGACTTCTAAGTGG,25,0.716001,0.875,0.833333,0.9,0.685775,0.734137,0.905797,0.990960,...,0,1,1,1,1,1,1,1,.........................,0.0


# IRESsite-based csv

In [21]:
dist_ss, dist_idx_ss = hamming_dist_idx(iressite_ss, gen_ss_1412)
dist_seq, dist_idx_seq = hamming_dist_idx(iressite_seq, gen_seq_1412)

nearest_ss_ires, nearest_seq_ires = [], []
for idx in range(len(iressite_seq)):
    nearest_ss_ires.append(dist_idx_ss[idx][1])
    nearest_seq_ires.append(dist_idx_seq[idx][1])
iressite_data['1412_NearestSS_IRESsite_IRES'] = nearest_ss_ires
iressite_data['1412_NearestSeq_IRESsite_IRES'] = nearest_seq_ires
iressite_data['1412_MinSS_IRESsite_IRES'] = dist_ss
iressite_data['1412_MinSeq_IRESsite_IRES'] = dist_seq

dist_ss, dist_idx_ss = hamming_dist_idx(iressite_ss, gen_ss_1410)
dist_seq, dist_idx_seq = hamming_dist_idx(iressite_seq, gen_seq_1410)
nearest_ss_ires, nearest_seq_ires = [], []
for idx in range(len(iressite_seq)):
    nearest_ss_ires.append(dist_idx_ss[idx][1])
    nearest_seq_ires.append(dist_idx_seq[idx][1])
iressite_data['1410_NearestSS_IRESsite_IRES'] = nearest_ss_ires
iressite_data['1410_NearestSeq_IRESsite_IRES'] = nearest_seq_ires
iressite_data['1410_MinSS_IRESsite_IRES'] = dist_ss
iressite_data['1410_MinSeq_IRESsite_IRES'] = dist_seq

iressite_data.to_csv('IRESsite_shorter300bp_similarGenSeq.csv')

100%|███████████████████████████████████████████| 81/81 [00:03<00:00, 22.22it/s]


In [22]:
iressite_data

,ID,Sequence,SS,MFE,1412_NearestSS_IRESsite_IRES,1412_NearestSeq_IRESsite_IRES,1412_MinSS_IRESsite_IRES,1412_MinSeq_IRESsite_IRES,1410_NearestSS_IRESsite_IRES,1410_NearestSeq_IRESsite_IRES,1410_MinSS_IRESsite_IRES,1410_MinSeq_IRESsite_IRES
0,IRESITEID472,AGAATTAGAATGTTTCTTAGCGGTCGTGTAGTTATTTTTATGTCAT...,((((..((((((((..((((.((((..............(((((((...,-25.900000,3087,2482,46,77,2623,2193,46,78
1,IRESITEID475,GGACAGTTTGGATAATTTTCCGTGAAACTGGATGATCTTTGAATAT...,...(((((((((......)))...))))))((((.(((((.........,-74.400002,5141,4831,108,139,3568,4786,110,139
2,IRESITEID592,CTTGCCAGCCAGTTGGGCAGCAGCAGGCAGTCCAGCCACAGGCCGT...,...(((.((((...((((((((.(((((((....(((.(..(((((...,-84.000000,4611,3537,60,104,4354,4502,63,98
3,IRESITEID335,CTGTGTTTGGCTCCACGCTCGATCCACTGGCGAGTGTTAGTAACAG...,..((((..((..(((...(((((.((((...(((((..(((((((....,-78.900002,4661,4181,68,102,3994,3527,67,102
4,IRESITEID336,CTGTGTTTGGCTCCACGCTCGATCCACTGGCGAGTGTTAGTAACAG...,..((((..((..(((...(((((.((((...(((((..(((((((....,-80.000000,4661,3409,68,103,3994,3527,67,102
...,...,...,...,...,...,...,...,...,...,...,...,...
76,IRESITEID425,AAGAAGAGGTAGCGAGTGGACGTGACTGCTCTATCCCGGGCAAAAG...,.......(((..((.((((((.(.(((((((((((((........)...,-124.300003,5224,4288,82,121,3774,4131,83,120
77,IRESITEID419,TGCAGAGATCCAGGGGAGGCGCCTGTGAGGCCCGGACCTGCCCCGG...,........(((.(((((((.((((...)))).(((..(((((..((...,-89.800003,3816,5282,76,110,3730,4296,76,107
78,IRESITEID418,CGGCCTGGAGTCTCCCAGTCTTGTCCCGGCAGTGCCGCCCTCCCCA...,.((.((((......)))).))..(((((((...)))((..((...(...,-30.700001,2218,1932,29,52,1956,1717,31,51
79,IRESITEID613,AAAAGAAGGAAAAAGAAGGAAAAGAAGGAAAAAGAAGGCTGCAGGC...,......................................((((((.....,-9.700000,1981,1326,20,47,1861,1178,20,46
